# Data Wrangling

In [9]:
import pandas as pd 
import plotly.express as px 
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.ar_model import AutoReg

**Importing Data**

In [34]:
def wrangle(filepath):
    #Read data into csv
    df = pd.read_csv(filepath)
    
    # Combine 'Year' and 'Month' into a 'Date' column
    df['Date'] = df['Month'] + ' ' + df["Year \xa0 \xa0 \xa0"].astype(str)

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%B %Y')

    # Set 'Date' as the index
    df.set_index('Date', inplace=True)

    # Sort the DataFrame by the new datetime index
    df.sort_index(inplace=True)

    # Keep only the 'Fraud Volumes (In Millions)' column
    fraud_series = df['Fraud Volumes (In Millions)']

    return fraud_series

In [36]:
wrangle(filepath="data/fraud_data.csv")

Date
2009-07-01    4288.5786
2009-08-01    4365.7617
2009-09-01    4222.9797
2009-10-01    4366.1658
2009-11-01    3143.0898
                ...    
2024-02-01    5971.9245
2024-03-01    6968.4351
2024-04-01    6289.2777
2024-05-01    6595.0467
2024-06-01    5760.5802
Name: Fraud Volumes (In Millions), Length: 180, dtype: float64